**실습에 필요한 libraries 들 설치 및 import**

In [1]:
!pip install mysql-connector-python

In [1]:
import mysql.connector
import sys
import pandas as pd
import matplotlib.pyplot as plt

**database 연결 정보 입력**

In [2]:
# Connect to the database
host = "147.46.15.238" # database 를 생성/연결할 서버 IP
port = "7000"
user = ""
password = ""
database = ""

# mysql 을 서버에서 활용할 수 있도록 연결해준다.
mysql_con = mysql.connector.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        database=database
    )

# Cursor 를 통해 서버의 database 와 커뮤니케이트 한다.
cur = mysql_con.cursor(dictionary=True) # dictionary

**load SQL Dump file**

get_dump 같은 경우, 처음 database 에 필요한 tables 생성시 호출하고 특이 케이스로 삭제되지 않는 한, 다시 호출할 필요없음.

In [3]:
# mysql_connector 로 mysql 서버를 호스트에 연결한 후 제공된 .sql file 을 읽어와 호스트에 database 를 생성하고 필요한 table 을 생성한다.

def get_dump(mysql_con, filename):

    query = ""
    try:
        with mysql_con.cursor() as cursor: # cursor 선언
            for line in open(filename, 'r', encoding='utf8'): # .sql 파일 열기
                if line.strip():
                    line = line.strip()
                    
                    # query 를 끝마치는 ";" 가 등장하기 전까지 파일을 읽으며 query 라는 변수에 저장 후 cursor 을 통해 서버의 database 로 송출
                    if line[-1] == ";":
                        query += line 
                        cursor.execute(query) # 서버의 database 로 보냄
                        query = ""
                    else:
                        query += line

    except Warning as warn:
        print(warn)
        sys.exit()

get_dump(mysql_con, 'ds2_database.sql')

In [ ]:
# test connection
query = "SHOW tables;"     # 서버의 database 에 보내고 싶은 sql query
cur.execute(query)         # cur.execute() 을 통해 query 를 서버로 송출
a = cur.fetchall()         # cur.fetchall() 을 통해 query 실행 후 결과값 반환
print(a)

In [5]:
'''
    SQL query 를 받아 해당 query를 보내고 그 결과 값을 dataframe으로 저장해 return 해주는 함수
'''
def get_output(query):
    cur.execute(query)
    out = cur.fetchall()
    df = pd.DataFrame(out)
    return df

**Connection Test**: 테이블에 들어있는 정보를 모두 확인하고 각 테이블의 사이즈를 확인한다. ".head()" 를 통해서 앞쪽 n개의 row 만 출력할 수 있다.

In [ ]:
# test connection
query = "SHOW COLUMNS FROM customer;"
get_output(query)

In [ ]:
query = "SELECT * FROM seller;"
get_output(query).head() # dataframe.head(n=5) : 앞쪽 n=5개 row 출력, SELECT * FROM seller LIMIT 5 와 동일

In [ ]:
query = "SELECT * FROM product;"
get_output(query).head()

In [ ]:
query = "SELECT * FROM geolocation;"
get_output(query).head()

In [ ]:
query = "SELECT * FROM order_info;"
get_output(query).head()

In [ ]:
query = "SELECT * FROM order_item;"
get_output(query).head()

In [ ]:
query = "SELECT * FROM order_payment;"
get_output(query).head()

In [ ]:
query = "SELECT * FROM order_review;"
get_output(query).head()

In [ ]:
query = "SELECT * FROM seller;"
get_output(query).head()

In [ ]:
# Table Size

query = "SELECT COUNT(customer_id) as num_customer FROM customer;"
print(get_output(query))

query = "SELECT COUNT(geolocation_zip_code_prefix) as num_geolocation FROM geolocation;"
print(get_output(query))

query = "SELECT COUNT(order_id) as num_order FROM order_info;"
print(get_output(query))

query = "SELECT COUNT(order_id) as num_item FROM order_item;"
print(get_output(query))

query = "SELECT COUNT(order_id) as num_pay FROM order_payment;"
print(get_output(query))

query = "SELECT COUNT(review_id) as num_review FROM order_review;"
print(get_output(query))

query = "SELECT COUNT(product_id) as num_product FROM product;"
print(get_output(query))

query = "SELECT COUNT(seller_id) as num_seller FROM seller;"
print(get_output(query))

**SQL Query Format in Python**

In [ ]:
"\ 를 사용해서 query format 을 맞춰준다."

# 예시)

# query = "SELECT *\
#          FROM (SELECT col1, col2\
#               FROM table1 JOIN table2 using (col3)\
#               WHERE condition\
#               GROUP BY col3\
#               HAVING condition)\
#          WHERE condition;"

Tips

- LIMIT : table 을 위에서 부터 원하는 row 수 만큼 잘라서 가져온다 
- 예시) SELECT * FROM customer LIMIT 5; \
       -> customer table 에서 레코드를 가져오는데 상위 5개만 가져온다

- SUM(condition) : table 의 특정 attribute 이 condition 과 부합할 시 1 로 리턴되고 SUM() 을 통해서 condition 과 부합하는 row 의 수를 리턴한다 
- 예시) SUM('product_category_name' = 'home_appliances') \
       -> product table 에서 'product_category_name' attribute 기준, 'home_appliances' 인 레코드들의 총 개수를 리턴한다

**Question 0-1**: customers 중 "rio" 로 시작되는 도시에 거주 중인 사람들의 unique_id 를 출력한다.


In [ ]:
query = ""

print(get_output(query))

**Question 0-2**: delivered 되지 않은 주문들의 order_id 및 order_status 를 출력한다.


In [ ]:
query = ""

print(get_output(query))

**Question 0-3**: 평점 3점 미만을 받은 주문들의 product_id 와 category 정보를 출력한다. 단, product_photos_qty 가 0 이 아닌 것들만 출력한다.

In [ ]:
query = ""

print(get_output(query))

**Question 1-1**: 가장 많은 주문량 (주문 건수, order_id 의 개수) 을 가지는 상품 카테고리 5종의 이름 (product_category_name) 및 각 카테고리에 대한 product_id 의 수 (num_prod_ids 로 지정, 중복 허용) 를 주문량 기준 내림차순으로 출력한다. Category 정보가 없는 상품은 포함하지 않는다. (내림차순으로 정렬시 num_prod_ids 가 같으면 상품 카테고리 이름의 오름차순으로 정렬한다.)

In [ ]:
query = ""
print(get_output(query))

**Question 1-2**: 주문량이 가장 적은 상품 카테고리 5종의 이름 및 각 카테고리에 속하는 product_id 의 수 (num_prods_ids 로 지정) 를 주문량에 대한 오름차순으로 출력한다. (오름차순으로 정렬시 num_prod_ids 가 같을 경우 상품 카테고리 이름의 오름차순으로 정렬한다)

In [ ]:
query = ""
print(get_output(query))

**Question 2**: order_info 테이블의 "order_purchase_timestamp" column 에는 상품 주문 날짜에 대한 정보가 (YYYY-MM-DD hh:mm:ss) 형태로 담겨있다. 상품 주문량을 월별로 합산하고, 월별 총 주문량을 그래프로 출력한다. (x-axis: MM, y-axis: num_orders)

In [ ]:
query = ""
df = get_output(query)
df.plot(kind='bar', x='', y='')

**Question 3**: 총 결제 금액이 가장 많은 customer에 대한 정보 (customer_unique_id, total_payment, total_orders, customer_city, customer_state) 를 총 결제 금액에 대한 내림차순으로 정렬 후, 상위 5명의 정보를 출력한다. 결제 금액이 동일한 경우 customer_unique_id 기준 사전 순서로 구별한다.

In [ ]:
query = ""
print(get_output(query))

**Question 4-1**: 가장 많은 매출을 기록한 도시 (total_sales 가 가장 높은 도시) 를 매출의 내림차순으로 정렬하고, 상위 5개 도시에 대한 정보(city, state, total_orders, total_revenue)를 출력한다. 매출 측정시 구매자 (구매자 위치) 를 기준으로 매출을 측정한다.

In [ ]:
query = ""
print(get_output(query))

**Question 4-2**: 가장 많은 매출을 기록한 주와 가장 적은 매출을 가지는 주의 이름 (max_state, min_state) 및 매출의 차를 순서대로 출력한다. 매출 측정시 구매자 (구매자 위치) 기준으로 매출을 측정한다.

In [ ]:
query = ""
print(get_output(query))

**Question 5**: 각 state (customer 기준) 마다 total_num_orders, num_cities, num_delivered, num_not_delivered, delivery_success_rate, delivery_failure_rate 를 출력한다. 성공적으로 배송된 주문은 항상 "delivered" 상태로 가정한다.

In [ ]:
query = ""
print(get_output(query))

**Question 6**: 가격이 가장 높은 아이템 10개의 product_id, price, category 를 가격의 내림차순으로 출력한다.

In [ ]:
query = ""
print(get_output(query))

**Question 7**: 한 세션 (timestamp 기준) 에 3 종류 이상의 아이템을 구매한 customer 의 주소 (city, state) 를 출력한다.

In [ ]:
query = ""
print(get_output(query))

**Question 8**: 

각각의 주문(order)에는 여러 아이템이 포함된다. 평점이 4점 이상이며 "delivered" 된 주문들을 선별해, 주문에 포함된 아이템들의 평균 price가 가장 높은 / 낮은 주문의 order_id 를 찾는다. 이 때, order의 평균 price가 같을 경우 order_id의 오름차순으로 선정한다. 

각 주문을 찾은 후 평균 price가 가장 높은 주문에서는 주문 내에서 price가 가장 높은 아이템의 product_id 와 price 를, 가장 낮은 주문에서는 price가 가장 낮은 아이템의 product_id와 price를 찾는다. 

하나의 query를 통해 앞선 과정을 모두 수행하고, order_id, product_id, product_price column을 가진 table을 얻는다. 첫번째 행에는 금액이 높은 주문, 두번째 행에는 금액이 낮은 주문의 정보를 표시한다.

In [ ]:
query = ""
print(get_output(query))